In [1]:
!pip install keras_vggface

  Running setup.py bdist_wheel for keras-vggface ... done
  Stored in directory: /root/.cache/pip/wheels/72/aa/01/eb7baeb2f6e2d2f0d2aabddb5f01d57fa22fbd019ee2799bf5
Successfully built keras-vggface
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import keras
from keras_vggface.vggface import VGGFace
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [3]:

# Get images
X = []
for filename in os.listdir('/input/Train/'):
    if filename != 'Thumbs.db':
        X.append(img_to_array(load_img('/input/Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X


#Load weights
#inception = InceptionResNetV2(weights='imagenet', include_top=True)
#inception.graph = tf.get_default_graph()

In [57]:
from keras_vggface import utils

# tensorflow
vgg_face_model = VGGFace(include_top=True, input_shape=(224, 224, 3))


In [59]:
# Change the image path with yours.
img = image.load_img('/input/Test/2004.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = utils.preprocess_input(x, version=1) # or version=2
preds = vgg_face_model.predict(x)
#print(preds)
print('Predicted:', utils.decode_predictions(preds))

Predicted: [[["b'Mary_Kay_Place'", 0.07033763], ["b'Juliet_Stevenson'", 0.046592996], ["b'Imelda_Staunton'", 0.027502062], ["b'Ruta_Lee'", 0.02171729], ["b'Adrianne_Curry'", 0.021684552]]]


In [60]:

vgg_face_model.graph = tf.get_default_graph()

In [62]:
inception.outputs

[<tf.Tensor 'predictions/Softmax:0' shape=(?, 1000) dtype=float32>]

In [61]:
print(vgg_face_model.outputs)


[<tf.Tensor 'fc8/softmax_4/Softmax:0' shape=(?, 2622) dtype=float32>]


58916864/58909280 [==============================] - 4s 0us/step


In [63]:
def conv_stack(data, filters, s):
        output = Conv2D(filters, (3, 3), strides=s, activation='relu', padding='same')(data)
        output = BatchNormalization()(output)
        return output

embed_input = Input(shape=(2622,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = conv_stack(encoder_input, 64, 2)
encoder_output = conv_stack(encoder_output, 128, 2)
encoder_output = conv_stack(encoder_output, 256, 2)
encoder_output = conv_stack(encoder_output, 512, 1)
encoder_output = conv_stack(encoder_output, 256, 1)

#Fusion
# y_mid: (None, 256, 28, 28)
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Permute((2, 1))(fusion_output) 
fusion_output = Reshape(([32, 32, 2622]))(fusion_output)
fusion_output = concatenate([fusion_output, encoder_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu')(fusion_output) 

#Decoder
decoder_output = UpSampling2D((2, 2))(fusion_output)
decoder_output = conv_stack(decoder_output, 128, 1)
decoder_output = conv_stack(decoder_output, 64, 1)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 32, 1)
decoder_output = conv_stack(decoder_output, 16, 1)
decoder_output = Conv2D(2, (2, 2), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [65]:
#Create embedding
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (224, 224, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with vgg_face_model.graph.as_default():
        embed = vgg_face_model.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 50

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

#Train model      
tensorboard = TensorBoard(log_dir="/output")
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=22, steps_per_epoch=40)

Epoch 1/22


/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


40/40 [==============================] - 187s 5s/step - loss: 0.0434
Epoch 2/22
40/40 [==============================] - 177s 4s/step - loss: 0.0114
Epoch 3/22
40/40 [==============================] - 177s 4s/step - loss: 0.0108
Epoch 4/22
40/40 [==============================] - 177s 4s/step - loss: 0.0106
Epoch 5/22
40/40 [==============================] - 177s 4s/step - loss: 0.0104
Epoch 6/22
40/40 [==============================] - 177s 4s/step - loss: 0.0102
Epoch 7/22
40/40 [==============================] - 177s 4s/step - loss: 0.0102
Epoch 8/22
40/40 [==============================] - 177s 4s/step - loss: 0.0100
Epoch 9/22
40/40 [==============================] - 177s 4s/step - loss: 0.0101
Epoch 10/22
40/40 [==============================] - 177s 4s/step - loss: 0.0099
Epoch 11/22
40/40 [==============================] - 176s 4s/step - loss: 0.0099
Epoch 12/22
40/40 [==============================] - 177s 4s/step - loss: 0.0098
Epoch 13/22
40/40 [=============================

In [ ]:



# Get images
imgs = glob.glob('*.jpg')

print('Found files:')
print(imgs)


for img in imgs:
    pic = cv2.imread(img, cv2.IMREAD_UNCHANGED)
    pic = cv2.resize(pic, (256, 256))
    cv2.imwrite(img, pic)

In [9]:
import glob
import os
import cv2

color_me = []
for filename in os.listdir('/input/Test/'):
    if filename != 'Thumbs.db':
        img = img_to_array(load_img('/input/Test/'+filename))
        img = cv2.resize(img,(256,256))
        color_me.append(img)


color_me = np.array(color_me, dtype=float)
color_me_embed = create_inception_embedding(color_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
	cur = np.zeros((256, 256, 3))
	cur[:,:,0] = color_me[i][:,:,0]
	cur[:,:,1:] = output[i]
	imsave("result/img_"+str(i)+".png", lab2rgb(cur))

/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/site-packages/skimage/color/colorconv.py:997: UserWarning: Color data out of range: Z < 0 in 3 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python3.6/site-packages/skimage/color/colorconv.py:997: UserWarning: Color data out of range: Z < 0 in 33 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python3.6/site-packages/skimage/color/colorconv.py:997: UserWarning: Color data out of range: Z < 0 in 2 pixels
  warn('Color data out of ra

In [ ]:



X = rgb2lab(1.0/255*X)[:,:,:,0]
X = np.array(X, dtype=float)
X_embed = create_inception_embedding(X)
# Test model
output = model.predict([X, X_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
	cur = np.zeros((256, 256, 3))
	cur[:,:,0] = X[i][:,:,0]
	cur[:,:,1:] = output[i]
	imsave("result/img_"+str(i)+".png", lab2rgb(cur))